# Course 3: Project - Part 2 - House prices

<a name="top"></a>
This notebook is concerned with Part 2 - House prices.

**Contents:**
* [Imports](#step-0)
* [Data cleaning](#step-1)

## Imports<a name="step-0"></a> ([top](#top))
---

In [1]:
# Standard library:
import collections
import json
import pathlib
import typing

# 3rd party:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

%matplotlib inline

## Data cleaning<a name="step-1"></a> ([top](#top))
---

In [2]:
df = pd.read_csv(pathlib.Path.cwd() / 'house-prices.csv')

We start by taking a look at the data set:

In [3]:
df.shape

(2430, 82)

In [4]:
with pd.option_context('display.max_columns', None):
    display(df.head())

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,484,528275070,60,RL,NaN,8795,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,7,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,300.0,Unf,0.0,652.0,952.0,GasA,Ex,Y,SBrkr,980,1276,0,2256,0.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,BuiltIn,2000.0,Fin,2.0,554.0,TA,TA,Y,224,54,0,0,0,0,NaN,NaN,NaN,0,4,2009,WD,Normal,236000
1,2586,535305120,20,RL,75.0,10170,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1951,1951,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,522.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,216.0,216.0,GasA,TA,Y,SBrkr,1575,0,0,1575,0.0,0.0,1,1,2,1,Gd,5,Typ,1,Gd,Attchd,1951.0,Unf,2.0,400.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal,155000
2,2289,923228250,160,RM,21.0,2001,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,4,5,1970,1970,Gable,CompShg,CemntBd,CmentBd,BrkFace,80.0,TA,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,546.0,546.0,GasA,Fa,Y,SBrkr,546,546,0,1092,0.0,0.0,1,1,3,1,TA,6,Typ,0,NaN,Attchd,1970.0,Unf,1.0,286.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,1,2007,WD,Normal,75000
3,142,535152150,20,RL,70.0,10552,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,5,1959,1959,Hip,CompShg,BrkFace,BrkFace,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,1018.0,Unf,0.0,380.0,1398.0,GasA,Gd,Y,SBrkr,1700,0,0,1700,0.0,1.0,1,1,4,1,Gd,6,Typ,1,Gd,Attchd,1959.0,RFn,2.0,447.0,TA,TA,Y,0,38,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,165500
4,2042,903475060,190,RM,60.0,10120,Pave,NaN,IR1,Bnk,AllPub,Inside,Gtl,OldTown,Feedr,Norm,2fmCon,2.5Unf,7,4,1910,1950,Hip,CompShg,Wd Sdng,Wd Sdng,None,0.0,Fa,TA,CBlock,TA,TA,No,Unf,0.0,Unf,0.0,925.0,925.0,GasA,TA,N,FuseF,964,925,0,1889,0.0,0.0,1,1,4,2,TA,9,Typ,1,Gd,Detchd,1960.0,Unf,1.0,308.0,TA,TA,N,0,0,264,0,0,0,NaN,MnPrv,NaN,0,1,2007,WD,Normal,122000


In [5]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2430 entries, 0 to 2429
Data columns (total 82 columns):
Order              2430 non-null int64
PID                2430 non-null int64
MS SubClass        2430 non-null int64
MS Zoning          2430 non-null object
Lot Frontage       2010 non-null float64
Lot Area           2430 non-null int64
Street             2430 non-null object
Alley              163 non-null object
Lot Shape          2430 non-null object
Land Contour       2430 non-null object
Utilities          2430 non-null object
Lot Config         2430 non-null object
Land Slope         2430 non-null object
Neighborhood       2430 non-null object
Condition 1        2430 non-null object
Condition 2        2430 non-null object
Bldg Type          2430 non-null object
House Style        2430 non-null object
Overall Qual       2430 non-null int64
Overall Cond       2430 non-null int64
Year Built         2430 non-null int64
Year Remod/Add     2430 non-null int64
Roof Style         24

### Step 1 - Find and handle incorrect and missing values
---

We are told that the data set contains incorrect and missing values. Our plan is:

* **Qualitative variables:** Make sur that they only take valid values. To that end, we have prepared a little JSON document that, for each nominal and ordinal variable, lists the valid values. (This document was generated by parsing the documentation. The code is in `qualitative_variables.py`.)

In [6]:
def count_null(series):
    return series.isna().sum()


def count_invalid(series, valid_values):
    return (~series.isin(valid_values)).sum()
    

def check_quantitative(series):
    print('null: {}'.format(count_null(series)))
    print(series.describe())

In [7]:
ql_fixes = []

#### Qualitative variables

We read the definitions of the qualitative variables:

In [8]:
def load_qualitative_variables(path):
    with open(path, 'r') as f:
        definitions = json.load(f)
        result = collections.OrderedDict()
        for definition in definitions:
            feature = definition['name']
            attrs = {
                'kind': definition['kind'],
                'values': set(definition['values'])
            }
            result[feature] = attrs
        return result

In [9]:
ql_vars = load_qualitative_variables('qualitative_variables.json')

# As per the description of the task, we will not check 'PID':
ql_vars.pop('PID')

print(f'qualitative variables: {len(ql_vars)}')

qualitative variables: 45


The 1st thing we notice when we try check the qualitative variables is that the names of some values differ between the data set and the documentation. *We decide to align the definitions to match the data set:*

In [10]:
def rename_var(cur_name, new_name):
    attrs = ql_vars.pop(cur_name)
    ql_vars[new_name] = attrs
    
    
# 'Exterior 1' is 'Exterior 1st' in the data set:
rename_var('Exterior 1', 'Exterior 1st')
# 'Exterior 2' is 'Exterior 2nd' in the data set:
rename_var('Exterior 2', 'Exterior 2nd')
# 'BsmtFinType 2' is 'BsmtFin Type 2' in the data set:
rename_var('BsmtFinType 2', 'BsmtFin Type 2')
# 'HeatingQC' is 'Heating QC' in the data set:
rename_var('HeatingQC', 'Heating QC')
# 'KitchenQual' is 'Kitchen Qual' in the data set:
rename_var('KitchenQual', 'Kitchen Qual')
# 'FireplaceQu' is 'Fireplace Qu' in the data set:
rename_var('FireplaceQu', 'Fireplace Qu')

The 2nd thing we notice is that _NA_ in the documentation is represented by `np.nan` in the data set. *We decide to align the definitions to match the data set:*

In [11]:
for feature, attrs in ql_vars.items():
    values = attrs['values']
    if 'NA' in values:
        print(f'fixing: {feature}')
        values.remove('NA')
        values.add(np.nan)

fixing: Alley
fixing: Bsmt Qual
fixing: Bsmt Cond
fixing: Bsmt Exposure
fixing: BsmtFin Type 1
fixing: Garage Type
fixing: Garage Qual
fixing: Garage Cond
fixing: Pool QC
fixing: Fence
fixing: Misc Feature
fixing: BsmtFin Type 2
fixing: Fireplace Qu


We can now check the qualitative variables and output a data-frame with the number of null values and the number of invalid values:

In [12]:
data = []
for feature, attrs in ql_vars.items():
    series = df[feature]
    null_count = count_null(series)
    invalid_count = count_invalid(series, attrs['values'])
    data.append((feature, attrs['kind'], null_count, invalid_count))
df_ql = pd.DataFrame(data=data, columns=['feature', 'kind', 'null_count', 'invalid_count'])

Here are the qualitative variables that we need to investigage:

In [13]:
df_ql[df_ql['invalid_count'] > 0]

,feature,kind,null_count,invalid_count
1,MS Zoning,Nominal,0,21
9,Neighborhood,Nominal,0,361
12,Bldg Type,Nominal,0,230
18,Mas Vnr Type,Nominal,20,20
28,Electrical,Ordinal,1,1
37,Sale Type,Nominal,0,2116
40,Exterior 2nd,Nominal,0,181


**MS Zoning:**

We look at the invalid values:

In [14]:
feature = 'MS Zoning'
values = ql_vars[feature]['values']
invalid = df.loc[~df[feature].isin(values), feature]
invalid.unique()

array(['I (all)', 'C (all)', 'A (agr)'], dtype=object)

We implement a correction and register it for later use:

In [15]:
def correct(df, ql_vars):
    feature = 'MS Zoning'
    values = ql_vars[feature]['values']
    # We only update invalid values:
    invalid = df.loc[~df[feature].isin(values), feature]
    corrected = invalid.map({'I (all)': 'I', 'C (all)': 'C', 'A (agr)': 'A'})
    df.loc[corrected.index, feature] = corrected
    return df


# Apply the correction:
df = correct(df, ql_vars)
assert count_invalid(df[feature], values) == 0, f"variable: '{feature}' not properly corrected"

# Register the correction:
ql_fixes.append(correct)

**Neighborhood (Nominal):**

We look at the invalid values:

In [16]:
feature = 'Neighborhood'
values = ql_vars[feature]['values']
invalid = df.loc[~df[feature].isin(values), feature]
invalid.unique()

array(['NAmes'], dtype=object)

Given the capitalization for `NWAmes`, we decide to align the definition to match the data set:

In [17]:
feature = 'Neighborhood'
values = ql_vars[feature]['values']
values.remove('Names')
values.add('NAmes')

assert count_invalid(df[feature], values) == 0, f"variable: '{feature}' not properly corrected"